<a href="https://colab.research.google.com/github/ManelSoengas/DL_Task1/blob/main/Data_augmetation_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nlpaug
!pip install transformers


In [ ]:
import nlpaug.augmenter.word as naw

# Ejemplo de DataFrame con columnas "text" y "language" que contiene sentencias en inglés y español
import pandas as pd
data = pd.DataFrame({
    'text': ['This is an English sentence.', 'Esto es una oración en español.'],
    'language': ['english', 'spanish']
})

# Crear instancias de aumentador de texto para inglés y español
aug_english = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")
aug_spanish = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-cased', action="insert")

# Aplicar aumentación de texto para cada idioma
augmented_texts = []
for idx, row in data.iterrows():
    if row['language'] == 'english':
        augmented_text = aug_english.augment(row['text'])
    elif row['language'] == 'spanish':
        augmented_text = aug_spanish.augment(row['text'])
    else:
        augmented_text = row['text']  # Mantener el texto sin cambios si el idioma no es inglés ni español
    augmented_texts.append(augmented_text)

# Agregar los textos aumentados al DataFrame
data['augmented_text'] = augmented_texts


In [ ]:
data.head()


,text,language,augmented_text
0,This is an English sentence.,english,[this is an early english conditional sentence.]
1,Esto es una oración en español.,spanish,[Esto es es una oración que en latín español.]


In [2]:
import pandas as pd
df_train = pd.read_csv('/content/EXIST2021_training.tsv', sep='\t')


In [ ]:
!pip install nlpaug
!pip install transformers

In [ ]:
import nlpaug.augmenter.word as naw
from transformers import BertTokenizer, BertForMaskedLM
import torch
import pandas as pd

# Cargar el DataFrame con las sentencias y las clases objetivo
# Supongamos que las columnas se llaman 'text' y 'language', y 'task1' contiene las clases objetivo
# Ejemplo de DataFrame con 4000 filas
# data = pd.read_csv('data.csv')

# Cargar el modelo BERT pre-entrenado multilingüe
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model.eval()

# Crear instancias de aumentador de texto para inglés y español
aug_english = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-cased', action="insert")
aug_spanish = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-cased', action="insert")

# Función para aplicar aumentación de texto a una oración
def augment_text(text, language):
    if language == 'en':
        augmented_text = aug_english.augment(text)
    elif language == 'es':
        augmented_text = aug_spanish.augment(text)
    else:
        augmented_text = text  # Mantener el texto sin cambios si el idioma no es inglés ni español
    return augmented_text

# Crear listas para almacenar las sentencias y clases objetivo aumentadas
augmented_texts = []
augmented_languages = []
classes = []

# Recorrer el DataFrame y aplicar aumentación a cada fila
for idx, row in df_train.iterrows():
    text = row['text']
    language = row['language']
    target_class = row['task1']

    # Aplicar aumentación de texto en el idioma correspondiente
    augmented_text_1 = augment_text(text, language)
    augmented_text_2 = augment_text(text, language)

    # Agregar las sentencias y clases objetivo aumentadas a las listas
    augmented_texts.extend([augmented_text_1, augmented_text_2])
    augmented_languages.extend([language, language])
    classes.extend([target_class, target_class])

# Crear un nuevo DataFrame con las sentencias y clases objetivo aumentadas
augmented_data = pd.DataFrame({
    'text': augmented_texts,
    'language': augmented_languages,
    'task1': classes
})

# Verificar el nuevo DataFrame con las sentencias y clases objetivo aumentadas
print(augmented_data.head())
